# Music Genre Recognition - Milestone 1
Darren Midkiff and Cheng-Wei Hu

## Overview
This project aims to identify the genre of music in an audio sample. Features will be extracted from the analog data, and a genre will be predicted using a KNN model, trained on labelled audio samples from the freely available FMA dataset.

## Training Data

The [FMA Dataset](https://github.com/mdeff/fma) is comprised of over 100,000 tracks from 161 genres. In order to make the problem more manageable, we will use the small version of the dataset, which includes 8,000 tracks from 8 top-level genres. The dataset also includes dozens of features -- year released, location of artist, number of listens, etc. Because this project aims to identify genre using only audio signal, all of these features are irrelevant and will be dropped.

In [76]:
import pandas as pd

#read full metadata file
metadata = pd.read_csv("./fma_metadata/tracks.csv", skiprows=[0,2], low_memory=False)

# drop all tracks that are not in fma_small dataset
metadata = metadata[metadata["subset"].eq("small")]
# add name to track_id column (missing because of stupid CSV formatting)
metadata = metadata.rename(columns={"Unnamed: 0": "track_id"})
# drop all columns that don't relate to genre
# we will not have this metadata from the audio file
metadata.drop(metadata.columns.difference(["track_id","genre_top"]),1,inplace=True)
# cast track id to integer
metadata = metadata.astype({'track_id':np.int32})
# reset indices accounting for dropped rows
metadata = metadata.reset_index(drop=True)

#write only relevant metadata to file for use in training
metadata.to_csv("fma_small_genres.csv")

metadata.head()

,track_id,genre_top
0,2,Hip-Hop
1,5,Hip-Hop
2,10,Pop
3,140,Folk
4,141,Folk


In [2]:
# show 8 genres
print(metadata["genre_top"].unique())

['Hip-Hop' 'Pop' 'Folk' 'Experimental' 'Rock' 'International' 'Electronic'
 'Instrumental']


## Feature Extraction

In order to apply machine learning techniques to audio samples, useful features must be extracted from the signals. We will extract four features: [Zero Crossing Rate](https://en.wikipedia.org/wiki/Zero-crossing_rate), [Spectral Centroid](https://en.wikipedia.org/wiki/Spectral_centroid), [Spectral Rolloff](https://en.wikipedia.org/wiki/Spectral_slope), and [Mel-Frequency Cepstral Coefficients](https://en.wikipedia.org/wiki/Mel-frequency_cepstrum).


In our project, we will use [librosa](https://librosa.org/doc/latest/index.html) to extract features from raw audio. Here, features are extracted from our training data and exported to a csv file.

In [18]:
%matplotlib inline
import librosa
import IPython.display as ipd
import matplotlib.pyplot as plt
import librosa.display
import sklearn
import numpy as np

In [4]:
# Get all the audio file paths from the fma_small dataset 
import os
file_names = []
for root, dirs, files in os.walk('./fma_small', topdown=False):
    for name in files:
        if name[-1] != '3':
            continue
        file_names.append(os.path.join(root, name))
# print(len(file_names))

In [49]:
# Define the function to extract the four features from  
def normalize(x, axis=0):
    return sklearn.preprocessing.minmax_scale(x, axis=axis)

def extract_feature_from_audio(audio_path, should_plot = False, should_print = False):
    # load
    x , sr = librosa.load(audio_path)
    if should_plot:
        plt.figure(figsize=(14, 5))
        librosa.display.waveplot(x, sr=sr)
    
    # zero_crossing_rate
    zero_crossing_rate = librosa.feature.zero_crossing_rate(x)
    m_zcr = np.mean(zero_crossing_rate) #mean zero-crossing rate
    v_zcr = np.var(zero_crossing_rate) #zero-crossing rate variance
    
    if should_print:
        print(zero_crossing_rate.shape)
        print(zero_crossing_rate)
        print(m_zcr)
    
    # spectral_centroids
    spectral_centroids = librosa.feature.spectral_centroid(x, sr=sr)[0]
    m_spec_cent = np.mean(spectral_centroids) #mean spectral centroid
    v_spec_cent = np.var(spectral_centroids) #spectral centroid variance
    
    if should_print:
        print(spectral_centroids.shape)
        print(spectral_centroids)
        print(m_spec_cent)

    # Computing the time variable for visualization
    # frames = range(len(spectral_centroids))
    # t = librosa.frames_to_time(frames)
    
    # spectral_rolloff
    spectral_rolloff = librosa.feature.spectral_rolloff(x, sr=sr)[0]
    m_spec_roll = np.mean(spectral_rolloff) #mean spectral rolloff
    v_spec_roll = np.var(spectral_rolloff) #spectral rolloff variance
    
    if should_print:
        print(spectral_rolloff.shape)
        print(spectral_rolloff)
        print(m_spec_roll)

    # mfccs
    mfccs = librosa.feature.mfcc(x, sr=sr)
    m_mfccs = np.mean(mfccs,axis=1) #mean mfccs
    v_mfccs = np.var(mfccs,axis=1) #mfcc variances
    
    if should_print:
        print(mfccs.shape)
        print(mfccs)
    #Displaying  the MFCCs:
    # librosa.display.specshow(mfccs, sr=sr, x_axis='time')
    
    return [m_zcr, v_zcr, m_spec_cent, v_spec_cent, m_spec_roll, v_spec_roll, m_mfccs, v_mfccs]

In [13]:
x,sr = librosa.load("./fma_small/000/000002.mp3")
zero_crossing_rate = librosa.feature.zero_crossing_rate(x)[0]
spectral_centroids = librosa.feature.spectral_centroid(x, sr=sr)[0]
spectral_rolloff = librosa.feature.spectral_rolloff(x, sr=sr)[0]
mfccs = librosa.feature.mfcc(x, sr=sr)

In [48]:
print(zero_crossing_rate.shape)
print(spectral_centroids.shape)
print(spectral_rolloff.shape)
print(mfccs.shape)

(1291,)
(1291,)
(1291,)
(20, 1291)


In [89]:
# Extract the features from audio files
# Some of the audio files are damaged. So we skipped those files.
# The damaged files are: './fma_small/099/099134.mp3', './fma_small/108/108925.mp3', './fma_small/133/133297.mp3'

train_audio_features_all = []
fail_file_names_all = []
fail_file_idx_all = []
fail_file_names_dict_all = {}

for idx, file in enumerate(file_names):
    print(idx, file)
    try:
        single_audio_features = extract_feature_from_audio(file)
        row = []
        row.append(int(file.split('\\')[-1].split('.')[0])) #extract integer file ID
        for f in single_audio_features:
            row.append(f)
        train_audio_features_all.append(row)
    except:
        print("Failed: ", idx, file)
        fail_file_names_all.append(file)
        fail_file_idx_all.append(file)
        fail_file_names_dict_all[file] = True

0 ./fma_small\000\000002.mp3


C:\Users\dclay\anaconda3\lib\site-packages\librosa\core\audio.py:162: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn("PySoundFile failed. Trying audioread instead.")


1 ./fma_small\000\000005.mp3
2 ./fma_small\000\000010.mp3
3 ./fma_small\000\000140.mp3
4 ./fma_small\000\000141.mp3
5 ./fma_small\000\000148.mp3
6 ./fma_small\000\000182.mp3
7 ./fma_small\000\000190.mp3
8 ./fma_small\000\000193.mp3
9 ./fma_small\000\000194.mp3
10 ./fma_small\000\000197.mp3
11 ./fma_small\000\000200.mp3
12 ./fma_small\000\000203.mp3
13 ./fma_small\000\000204.mp3
14 ./fma_small\000\000207.mp3
15 ./fma_small\000\000210.mp3
16 ./fma_small\000\000211.mp3
17 ./fma_small\000\000212.mp3
18 ./fma_small\000\000213.mp3
19 ./fma_small\000\000255.mp3
20 ./fma_small\000\000256.mp3
21 ./fma_small\000\000368.mp3
22 ./fma_small\000\000424.mp3
23 ./fma_small\000\000459.mp3
24 ./fma_small\000\000534.mp3
25 ./fma_small\000\000540.mp3
26 ./fma_small\000\000546.mp3
27 ./fma_small\000\000574.mp3
28 ./fma_small\000\000602.mp3
29 ./fma_small\000\000615.mp3
30 ./fma_small\000\000620.mp3
31 ./fma_small\000\000621.mp3
32 ./fma_small\000\000625.mp3
33 ./fma_small\000\000666.mp3
34 ./fma_small\000\

269 ./fma_small\006\006330.mp3
270 ./fma_small\006\006331.mp3
271 ./fma_small\006\006332.mp3
272 ./fma_small\006\006333.mp3
273 ./fma_small\006\006342.mp3
274 ./fma_small\006\006354.mp3
275 ./fma_small\006\006357.mp3
276 ./fma_small\006\006358.mp3
277 ./fma_small\006\006360.mp3
278 ./fma_small\006\006363.mp3
279 ./fma_small\006\006366.mp3
280 ./fma_small\006\006367.mp3
281 ./fma_small\006\006368.mp3
282 ./fma_small\006\006370.mp3
283 ./fma_small\006\006372.mp3
284 ./fma_small\006\006373.mp3
285 ./fma_small\006\006376.mp3
286 ./fma_small\006\006379.mp3
287 ./fma_small\006\006380.mp3
288 ./fma_small\006\006381.mp3
289 ./fma_small\006\006382.mp3
290 ./fma_small\006\006383.mp3
291 ./fma_small\006\006385.mp3
292 ./fma_small\006\006387.mp3
293 ./fma_small\006\006389.mp3
294 ./fma_small\006\006390.mp3
295 ./fma_small\006\006393.mp3
296 ./fma_small\006\006394.mp3
297 ./fma_small\006\006396.mp3
298 ./fma_small\006\006406.mp3
299 ./fma_small\006\006407.mp3
300 ./fma_small\006\006439.mp3
301 ./fm

534 ./fma_small\011\011786.mp3
535 ./fma_small\011\011787.mp3
536 ./fma_small\011\011788.mp3
537 ./fma_small\011\011789.mp3
538 ./fma_small\011\011790.mp3
539 ./fma_small\011\011791.mp3
540 ./fma_small\011\011792.mp3
541 ./fma_small\011\011793.mp3
542 ./fma_small\011\011794.mp3
543 ./fma_small\011\011795.mp3
544 ./fma_small\011\011803.mp3
545 ./fma_small\011\011818.mp3
546 ./fma_small\011\011839.mp3
547 ./fma_small\011\011861.mp3
548 ./fma_small\011\011862.mp3
549 ./fma_small\011\011867.mp3
550 ./fma_small\011\011868.mp3
551 ./fma_small\011\011916.mp3
552 ./fma_small\011\011917.mp3
553 ./fma_small\011\011918.mp3
554 ./fma_small\011\011919.mp3
555 ./fma_small\011\011920.mp3
556 ./fma_small\011\011921.mp3
557 ./fma_small\011\011922.mp3
558 ./fma_small\011\011933.mp3
559 ./fma_small\011\011937.mp3
560 ./fma_small\011\011942.mp3
561 ./fma_small\011\011946.mp3
562 ./fma_small\011\011947.mp3
563 ./fma_small\011\011951.mp3
564 ./fma_small\012\012045.mp3
565 ./fma_small\012\012046.mp3
566 ./fm

...

6967 ./fma_small\133\133333.mp3
6968 ./fma_small\133\133431.mp3
6969 ./fma_small\133\133432.mp3
6970 ./fma_small\133\133433.mp3
6971 ./fma_small\133\133434.mp3
6972 ./fma_small\133\133435.mp3
6973 ./fma_small\133\133436.mp3
6974 ./fma_small\133\133437.mp3
6975 ./fma_small\133\133438.mp3
6976 ./fma_small\133\133439.mp3
6977 ./fma_small\133\133440.mp3
6978 ./fma_small\133\133441.mp3
6979 ./fma_small\133\133442.mp3
6980 ./fma_small\133\133443.mp3
6981 ./fma_small\133\133444.mp3
6982 ./fma_small\133\133445.mp3
6983 ./fma_small\133\133446.mp3
6984 ./fma_small\133\133447.mp3
6985 ./fma_small\133\133448.mp3
6986 ./fma_small\133\133449.mp3
6987 ./fma_small\133\133450.mp3
6988 ./fma_small\133\133451.mp3
6989 ./fma_small\133\133452.mp3
6990 ./fma_small\133\133453.mp3
6991 ./fma_small\133\133454.mp3
6992 ./fma_small\133\133455.mp3
6993 ./fma_small\133\133456.mp3
6994 ./fma_small\133\133457.mp3
6995 ./fma_small\133\133459.mp3
6996 ./fma_small\133\133479.mp3
6997 ./fma_small\133\133535.mp3
6998 ./f

7224 ./fma_small\137\137423.mp3
7225 ./fma_small\137\137424.mp3
7226 ./fma_small\137\137425.mp3
7227 ./fma_small\137\137426.mp3
7228 ./fma_small\137\137462.mp3
7229 ./fma_small\137\137463.mp3
7230 ./fma_small\137\137484.mp3
7231 ./fma_small\137\137500.mp3
7232 ./fma_small\137\137551.mp3
7233 ./fma_small\137\137561.mp3
7234 ./fma_small\137\137563.mp3
7235 ./fma_small\137\137567.mp3
7236 ./fma_small\137\137593.mp3
7237 ./fma_small\137\137605.mp3
7238 ./fma_small\137\137624.mp3
7239 ./fma_small\137\137627.mp3
7240 ./fma_small\137\137630.mp3
7241 ./fma_small\137\137631.mp3
7242 ./fma_small\137\137632.mp3
7243 ./fma_small\137\137715.mp3
7244 ./fma_small\137\137716.mp3
7245 ./fma_small\137\137717.mp3
7246 ./fma_small\137\137719.mp3
7247 ./fma_small\137\137720.mp3
7248 ./fma_small\137\137721.mp3
7249 ./fma_small\137\137722.mp3
7250 ./fma_small\137\137723.mp3
7251 ./fma_small\137\137724.mp3
7252 ./fma_small\137\137725.mp3
7253 ./fma_small\137\137740.mp3
7254 ./fma_small\137\137895.mp3
7255 ./f

7481 ./fma_small\141\141903.mp3
7482 ./fma_small\141\141972.mp3
7483 ./fma_small\142\142078.mp3
7484 ./fma_small\142\142079.mp3
7485 ./fma_small\142\142080.mp3
7486 ./fma_small\142\142081.mp3
7487 ./fma_small\142\142082.mp3
7488 ./fma_small\142\142083.mp3
7489 ./fma_small\142\142084.mp3
7490 ./fma_small\142\142085.mp3
7491 ./fma_small\142\142086.mp3
7492 ./fma_small\142\142087.mp3
7493 ./fma_small\142\142088.mp3
7494 ./fma_small\142\142089.mp3
7495 ./fma_small\142\142091.mp3
7496 ./fma_small\142\142092.mp3
7497 ./fma_small\142\142093.mp3
7498 ./fma_small\142\142094.mp3
7499 ./fma_small\142\142096.mp3
7500 ./fma_small\142\142097.mp3
7501 ./fma_small\142\142098.mp3
7502 ./fma_small\142\142128.mp3
7503 ./fma_small\142\142129.mp3
7504 ./fma_small\142\142132.mp3
7505 ./fma_small\142\142133.mp3
7506 ./fma_small\142\142358.mp3
7507 ./fma_small\142\142359.mp3
7508 ./fma_small\142\142360.mp3
7509 ./fma_small\142\142361.mp3
7510 ./fma_small\142\142362.mp3
7511 ./fma_small\142\142381.mp3
7512 ./f

7738 ./fma_small\145\145746.mp3
7739 ./fma_small\145\145747.mp3
7740 ./fma_small\145\145748.mp3
7741 ./fma_small\145\145749.mp3
7742 ./fma_small\145\145750.mp3
7743 ./fma_small\145\145751.mp3
7744 ./fma_small\145\145752.mp3
7745 ./fma_small\145\145754.mp3
7746 ./fma_small\145\145755.mp3
7747 ./fma_small\145\145756.mp3
7748 ./fma_small\145\145757.mp3
7749 ./fma_small\145\145758.mp3
7750 ./fma_small\145\145759.mp3
7751 ./fma_small\145\145760.mp3
7752 ./fma_small\145\145761.mp3
7753 ./fma_small\145\145762.mp3
7754 ./fma_small\145\145777.mp3
7755 ./fma_small\145\145780.mp3
7756 ./fma_small\145\145783.mp3
7757 ./fma_small\145\145887.mp3
7758 ./fma_small\145\145917.mp3
7759 ./fma_small\145\145918.mp3
7760 ./fma_small\146\146017.mp3
7761 ./fma_small\146\146018.mp3
7762 ./fma_small\146\146019.mp3
7763 ./fma_small\146\146020.mp3
7764 ./fma_small\146\146070.mp3
7765 ./fma_small\146\146147.mp3
7766 ./fma_small\146\146148.mp3
7767 ./fma_small\146\146149.mp3
7768 ./fma_small\146\146150.mp3
7769 ./f

7995 ./fma_small\154\154308.mp3
7996 ./fma_small\154\154309.mp3
7997 ./fma_small\154\154413.mp3
7998 ./fma_small\154\154414.mp3
7999 ./fma_small\155\155066.mp3


## Adding Genre Labels

The extracted features are combined with the labels and written to a .csv

In [97]:
m_mfccs.shape

(20,)

In [100]:
df['mfccs_mean'].shape

(7997,)

In [93]:
#put features in a pandas dataframe
feature_names = ["track_id", "zcr_mean", "zcr_var", "spec_cent_mean", "spec_cent_var", 
                 "spec_roll_mean", "spec_roll_var", "mfccs_mean", "mfccs_var"]
df = pd.DataFrame(train_audio_features_all, columns = feature_names)

#add genre labels from metadata
df = pd.merge(left=df, right=metadata, how='left', left_on='track_id', right_on='track_id')

df.head()

,track_id,zcr_mean,zcr_var,spec_cent_mean,spec_cent_var,spec_roll_mean,spec_roll_var,mfccs_mean,mfccs_var,genre_top
0,2,0.163695,0.008725,3037.644725,9.809958e+05,6214.647549,2.281126e+06,"[-67.3842, 65.155075, -10.669196, 12.176932, -...","[2907.9536, 927.7224, 519.6038, 312.80936, 306...",Hip-Hop
1,5,0.100023,0.004433,2414.124259,7.569450e+05,5437.441882,2.535670e+06,"[-106.58773, 87.23936, 12.156827, 24.140398, -...","[2197.3853, 843.6833, 457.25446, 314.9391, 276...",Hip-Hop
2,10,0.148674,0.000798,2349.559292,1.419561e+05,4315.856035,6.900174e+05,"[-17.238174, 94.44203, -48.614548, 32.370766, ...","[797.04956, 417.18228, 307.51672, 135.75954, 1...",Pop
3,140,0.044322,0.002694,1746.322280,1.049483e+06,3863.175065,5.928252e+06,"[-278.57538, 101.53074, 36.42724, 35.59012, 26...","[7312.72, 1265.0995, 471.97086, 237.86455, 195...",Folk
4,141,0.061836,0.002676,1654.237111,6.403904e+05,3315.637916,2.762245e+06,"[-208.56926, 116.633934, -1.7237115, 26.018599...","[12825.669, 1198.2709, 278.61575, 253.6174, 20...",Folk


In [101]:
df.to_pickle('./labels_features.pkl')

The exported csv file can be found [here](https://drive.google.com/file/d/1xiyYZ23WVl-RyWIWlf3-JZaZll4i9b1X/view?usp=sharing).